# Project Setup 🛠

## include setup.py:

In [ ]:
from setuptools import find_packages
from setuptools import setup

with open("requirements.txt") as f:
    content = f.readlines()
requirements = [x.strip() for x in content if "git+" not in x]

setup(name='uk_road_safety',
      version="0.0.1",
      description="UK road safety Model (api_pred)",
      install_requires=requirements,
      packages=find_packages(),
      test_suite="tests",
      # include_package_data: to install data from MANIFEST.in
      include_package_data=True,
      zip_safe=False)

## .env file:

In [ ]:
LOCAL_DATA_PATH=~/code/MagicaBleps/UK-road-safety/raw_data/data
LOCAL_REGISTRY_PATH=~/code/MagicaBleps/UK-road-safety/raw_data/training_outputs


#Docker
IMAGE=ukroadsafety
#GCP
PROJECT=wagon-bootcamp-365515
GCR_MULTI_REGION=eu.gcr.io
MEMORY=2Gi
REGION=europe-west1


## requirements.txt file:

In [ ]:
pygeohash
pandas
numpy
matplotlib
seaborn
#tensorflow

# API
fastapi         # API framework
pytz            # time zone management
uvicorn         # web server
# tests
httpx           # HTTP client
pytest-asyncio  # asynchronous I/O support for pytest

## add to Makefile to test Fastapi locally

In [ ]:
run_api:
	uvicorn uk_road_safety.API.api:app --reload

## add __init__.py to the root of def folder and empty __init__.py to subfolder.

In [ ]:
from os.path import isfile
from os.path import dirname

version_file = '{}/version.txt'.format(dirname(__file__))

if isfile(version_file):
    with open(version_file) as version_file:
        __version__ = version_file.read().strip()


# Build the API 📡

After project Setup, create api file:

In [ ]:
from datetime import datetime
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import pandas as pd
import numpy as np
import tensorflow as tf
import json
from fastapi.responses import HTMLResponse

app = FastAPI()


app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allows all origins
    allow_credentials=True,
    allow_methods=["*"],  # Allows all methods
    allow_headers=["*"],  # Allows all headers
)

@app.get("/predict")
def predict(hash):
    model = tf.keras.models.load_model(f'model/{hash}')
    X = np.load(f'data/X_{hash}.npy')
    y_pred = np.round(np.array(model.predict(X))).reshape(6,1)
    predictions={}
    months={0:'1_January',1:'2_February',2:'3_March',3:'4_April',4:'5_May',5:'6_June'}
    for i,p in enumerate(y_pred):
        predictions[months[i]]=int(p[0])
    return predictions

@app.get("/show_map")
def show_map(year):
    filename= 'map/'+str(year)+'.html'
    with open(filename, "r", encoding='utf-8') as f:
        html_content=f.read()
    return HTMLResponse(content=html_content, status_code=200)

@app.get("/")
def root():
   return {'greeting': 'Hello, We are team 6: UK road Safety'}

Now, you can run the API locally with the code in command: make run_api

http://localhost:8000


http://localhost:8000/docs

# Build a Docker Image for our API 🐳

## Create Dockerfile:

In [ ]:
#FROM python:3.10.6-slim
FROM tensorflow/tensorflow:2.11.0
COPY . .
RUN pip install --upgrade pip
RUN pip install .
CMD uvicorn uk_road_safety.API.api:app --host 0.0.0.0 --port $PORT

💻 Choose a Docker image name and add it to your .env

In [ ]:
IMAGE=ukroadsafety

💻 Once built, the image should be visible in the list of images built with the following command:

In [ ]:
docker images

Check the API Image

In [ ]:
docker run -it -e PORT=8000 -p 8000:8000 $IMAGE:dev sh

Exit the terminal and stop the container at any moment with:

In [ ]:
exit

## Run the API Image

In [ ]:
docker run -e PORT=8000 -p 8000:8000 --env-file .env $IMAGE:dev

## Stop the API Image

First, run the code: docker ps

Second, copy the image id: XXXXXXXXXXX

Third, run the code: docker stop xxxxxxxxxx. to stop the docker image.

## Deploy the API 🌎

Change your base image [Only for Intel processor users]

💻 Update your Dockerfile base image with either tensorflow/tensorflow:2.11.0 (if you are on an Intel processor only)

💻 Remove tensorflow from your requirements.txt because it is now pre-build with the image.

## Setup

First, let’s make sure to enable the Google Container Registry API for your project in GCP.

https://console.cloud.google.com/apis/enableflow?apiid=containerregistry.googleapis.com&redirect=https:%2F%2Fcloud.google.com%2Fcontainer-registry%2Fdocs%2Fquickstart&project=wagon-bootcamp-365515


Once this is done, let’s allow the docker command to push an image to GCP.

In [ ]:
gcloud auth configure-docker

Build and Push the Image to GCR

Add a GCR_MULTI_REGION variable to your project configuration and set it to eu.gcr.io.

In [ ]:
docker build -t $GCR_MULTI_REGION/$PROJECT/$IMAGE:prod .

Again, let’s make sure that our image runs correctly, so as to avoid wasting time pushing a broken image to the cloud.

In [ ]:
docker run -e PORT=8000 -p 8000:8000 --env-file .env $GCR_MULTI_REGION/$PROJECT/$IMAGE:prod

Visit http://localhost:8000/ and check whether the API is running as expected.

We can now push our image to Google Container Registry.

In [ ]:
docker push $GCR_MULTI_REGION/$PROJECT/$IMAGE:prod

The image should be visible in the GCP console.

https://console.cloud.google.com/gcr/images/wagon-bootcamp-365515?project=wagon-bootcamp-365515

## Deploy the Container Registry Image to Google Cloud Run

Add a MEMORY variable to your project configuration and set it to 2Gi.

👉 This will allow your container to run with 2GiB (= Gibibyte) of memory

💻 Create a .env.yaml file containing all the necessary environment variables

In [ ]:
LOCAL_DATA_PATH: ~/code/MagicaBleps/UK-road-safety/raw_data/data
LOCAL_REGISTRY_PATH: ~/code/MagicaBleps/UK-road-safety/raw_data/training_outputs


#Docker
IMAGE: ukroadsafety
#GCP
PROJECT: wagon-bootcamp-365515
GCR_MULTI_REGION: eu.gcr.io
MEMORY: 2Gi
REGION: europe-west1

💻 Using the gcloud run deploy --help documentation, identify a parameter that allows you to pass environment variables to your container on deployment

In [ ]:
gcloud run deploy --env-vars-file .env.yaml

Let’s run one last command 🤞

In [ ]:
gcloud run deploy --image $GCR_MULTI_REGION/$PROJECT/$IMAGE:prod --memory $MEMORY --region $REGION --env-vars-file .env.yaml

After confirmation, you should see something like this, indicating that the service is live 🎉

Service name (wagon-data-tpl-image):

Allow unauthenticated invocations to [wagon-data-tpl-image] (y/N)?  y

Deploying container to Cloud Run service [wagon-data-tpl-image] in project 
[le-wagon-data] region [europe-west1]

✓ Deploying new service... Done.

  ✓ Creating Revision... Revision deployment finished. Waiting for health 
  check to begin.

  ✓ Routing traffic...
  
  ✓ Setting IAM Policy...

Done.

Service [wagon-data-tpl-image] revision [wagon-data-tpl-image-00001-kup] has 
been deployed and is serving 100 percent of traffic.

Service URL: https://wagon-data-tpl-image-xi54eseqrq-ew.a.run.app

Any developer in the world 🌍 is now able to browse to the deployed url and get a prediction using the API 🤖!